# Keyword querying and filtering

<a target="_blank" href="https://colab.research.google.com/github/elasticsearch-labs/blob/main/search/01-keyword-querying-filtering.ipynb"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

이 대화형 노트북은 공식 Elasticsearch Python 클라이언트를 사용하여 기본 Elasticsearch 쿼리를 소개합니다. 이 섹션을 시작하기 전에 [빠른 시작](https://github.com/elastic/elasticsearch-labs/blob/main/notebooks/search/00-quick-start.ipynb)을 통해 작업하는 것이 좋습니다.

# Install and import libraries

In [ ]:
!pip install elasticsearch

In [ ]:

from elasticsearch import Elasticsearch
import pandas as pd


# Create the client instance


In [ ]:
from getpass import getpass

ES_URL = getpass('Elasticsearch URL: ')
ES_ID = "elastic" 
ES_PW = getpass('elastic user PW: ')
CERT_PATH = getpass('Elasticsearch cer 파일 경로: ')

# Create the client instance
client = Elasticsearch(
    ES_URL,
    basic_auth=(ES_ID, ES_PW),
    ca_certs=CERT_PATH
)

# Pretty print the response
def pretty_response_transform(response):
    result = []
    for hit in response['hits']['hits']:
        result.append({
            'id' : hit['_id'],
            'publication_date' : hit['_source']['publish_date'],
            'score' : hit['_score'],
            'title' : hit['_source']['title'],
            'summary' : hit['_source']['summary']
        })
    return result

## Querying

쿼리 컨텍스트에서 쿼리 절은 "이 문서가 이 쿼리 절과 얼마나 잘 일치합니까?"라는 질문에 대답합니다. 문서가 일치하는지 여부를 결정하는 것 외에도 쿼리 절은 `_score` 메타데이터 필드에서 관련성 점수도 계산합니다.

### Full text queries

전체 텍스트 쿼리를 사용하면 이메일 본문과 같은 분석된 텍스트 필드를 검색할 수 있습니다. 쿼리 문자열은 인덱싱 중에 필드에 적용된 것과 동일한 분석기를 사용하여 처리됩니다.

* **match**.   
    퍼지 일치(fuzzy matching), 구문(phrase) 또는 근접(proximity) 쿼리를 포함하여 전체 텍스트 쿼리를 수행하기 위한 표준 쿼리입니다.
* **multi-match**.   
    match query의 다중 필드 버전입니다.

### Match query
제공된 텍스트, 숫자, 날짜 또는 부울 값과 `match` 하는 문서를 반환합니다. 제공된 텍스트는 일치 전에 분석됩니다.

`match` 쿼리는 유사 항목 일치 옵션을 포함하여 전체 텍스트 검색을 수행하기 위한 표준 쿼리입니다.

[자세히 알아보기](https://www.elastic.co/guide/en/elasticsearch/reference/current/query-dsl-match-query.html#match-query-ex-request).


In [ ]:
response = client.search(index="book_index", query={
    "match": {
        "summary": {
            "query": "guide"
            }
        }
    })

pd.DataFrame.from_records(pretty_response_transform(response))

### Multi-match query

`multi_match` 쿼리는 일치 쿼리를 기반으로 다중 필드 쿼리를 허용합니다.

[Read more](https://www.elastic.co/guide/en/elasticsearch/reference/current/query-dsl-multi-match-query.html).

In [ ]:
response = client.search(index="book_index", query={
    "multi_match": {
        "query": "javascript",
        "fields": ["summary", "title"]
        }
    })

pd.DataFrame.from_records(pretty_response_transform(response))

캐럿(^) 표기법을 사용하여 개별 필드를 강화할 수 있습니다.

In [ ]:
response = client.search(index="book_index", query={
    "multi_match": {
        "query": "javascript",
        "fields": ["summary", "title^3"]
        }
    })

pd.DataFrame.from_records(pretty_response_transform(response))

### Prefix search

제공된 필드에 특정 접두사가 포함된 문서를 반환합니다.

[Read more](https://www.elastic.co/guide/en/elasticsearch/reference/current/query-dsl-prefix-query.html)

In [ ]:
response = client.search(index="book_index", query={
    "prefix": {
        "title": {
            "value": 'java'
            }
        }
    })

pd.DataFrame.from_records(pretty_response_transform(response))

### Fuzzy search

Levenshtein 편집 거리로 측정된 검색어와 유사한 용어가 포함된 문서를 반환합니다.

편집 거리는 한 용어를 다른 용어로 바꾸는 데 필요한 한 문자 변경 횟수입니다. 이러한 변경 사항에는 다음이 포함될 수 있습니다.

* 캐릭터 변경 (box → fox)
* 캐릭터 삭제 (black → lack)
* 문자 삽입 (sic → sick)
* 인접한 두 문자 바꾸기 (act → cat)

[Read more](https://www.elastic.co/guide/en/elasticsearch/reference/current/query-dsl-fuzzy-query.html)



In [ ]:
response = client.search(index="book_index", query={
    "fuzzy": {
        "title": {
            "value": 'pyvascript'
            }
        }
    })

pd.DataFrame.from_records(pretty_response_transform(response))

## Filtering

필터 컨텍스트에서 쿼리 절은 *"이 문서가 이 쿼리 절과 일치합니까?"*라는 질문에 대답합니다. 대답은 간단하게 예 또는 아니요 입니다. — 점수는 계산되지 않습니다. 필터 컨텍스트는 구조화된 데이터를 필터링하는 데 주로 사용됩니다. 예를 들면 다음과 같습니다.
* 이 `timestamp`는 2015년부터 2016년까지의 범위에 속합니까?
* `status` 필드가 `"published"`으로 설정되어 있나요?

필터 컨텍스트는 `bool` 쿼리의 `filter` 또는 `must_not` 매개변수와 같은 `filter` 매개변수에 쿼리 절이 전달될 때마다 적용됩니다.

[Read more](https://www.elastic.co/guide/en/elasticsearch/reference/current/query-dsl-bool-query.html)

### **bool.must**

절(쿼리)은 일치하는 문서에 나타나야 하며 점수에 영향을 미칩니다.

In [ ]:
response = client.search(index="book_index", query={
    "bool": {
        "must": [{
            "term": {
                "summary": "guide"
                }
            }, {
            "term": {
                "summary": "code"
                }
            }]
        }
    })

pd.DataFrame.from_records(pretty_response_transform(response))

### **bool.should**

해당 절(쿼리)이 일치하는 문서에 나타나야 합니다.

In [ ]:
response = client.search(index="book_index", query={
    "bool": {
        "should": [{
            "term": {
                "summary": "guide"
                }
            }, {
              "term": {
                  "summary": "code"
                  }
            }]
        }
    })

pd.DataFrame.from_records(pretty_response_transform(response))

### **bool.filter**

절(쿼리)은 일치하는 문서에 나타나야 합니다. **그러나 `must`와 달리 쿼리의 `score`는 무시됩니다.** 필터 절은 필터 컨텍스트에서 실행됩니다. 즉, 점수가 무시되고 절이 캐싱 대상으로 고려됩니다.

In [ ]:
response = client.search(index="book_index", query={
    "bool": {
        "filter": [{
            "term": {
                "summary": "guide"
                }
            }]
        }
    })

pd.DataFrame.from_records(pretty_response_transform(response))